In [ ]:
def generate_metric_table(df, athlete_id, selected_metrics, unit_dict, pdf, custom_labels):
    """
    Generate a clean, easy-to-read table summarizing selected metrics with Onboarding, Most Recent, PR values, and dates/phases.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        athlete_id (int): Athlete ID.
        selected_metrics (list): List of tuples specifying metrics to include.
        unit_dict (dict): Dictionary mapping (Test Type, Test Sub-Type) to units.
        pdf (PdfPages): PDF object to save the plot.
        custom_labels (list): List of custom labels corresponding to selected metrics.

    Returns:
        None
    """
    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        print(f"No data found for athlete ID {athlete_id}.")
        return

    # Initialize table data
    table_data = []
    column_headers = ["Metric", "Onboarding", "Most Recent", "PR"]

    for idx, (test_type, test_sub_type, metric) in enumerate(selected_metrics):
        metric_label = custom_labels[idx]
        metric_data = athlete_data[
            (athlete_data['Test Type'] == test_type) & (athlete_data['Test Sub-Type'] == test_sub_type)
        ]

        if metric_data.empty:
            onboarding_value = most_recent_value = pr_value = "N/A"
            onboarding_info = most_recent_info = pr_info = "N/A"
        else:
            if metric in metric_data.columns:
                metric_data = metric_data.copy()
                metric_data[metric] = pd.to_numeric(metric_data[metric], errors='coerce')

            def extract_value_info(data):
                """ Extract metric value and corresponding date/phase info """
                if not data.empty and metric in data.columns:
                    value = data[metric].iloc[0]

                    # Ensure the 'Date' column is properly formatted as datetime
                    if 'Date' in data.columns:
                        data = data.copy()
                        data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

                    date_phase = f"{data['Date'].iloc[0].strftime('%m/%d/%y')}" if pd.notna(data['Date'].iloc[0]) else "N/A"

                    if 'Phase' in data.columns and pd.notna(data['Phase'].iloc[0]):
                        date_phase += f", {data['Phase'].iloc[0]}"

                    return value, date_phase

                return "N/A", "N/A"

            # Extract values for Onboarding, Most Recent, PR
            onboarding_value, onboarding_info = extract_value_info(metric_data[metric_data['Test Number'] == 1])
            most_recent_value, most_recent_info = extract_value_info(
                metric_data[metric_data['Test Number'] == metric_data['Test Number'].max()]
            )
            pr_value = metric_data[metric].max() if metric in metric_data.columns else "N/A"
            pr_info = extract_value_info(metric_data[metric_data[metric] == pr_value])[1] if pr_value != "N/A" else "N/A"

        # Get units for the metric
        units = unit_dict.get((test_type, test_sub_type), "")

        def format_with_units(value):
            return f"{value} {units}" if value != "N/A" and units else value

        # Format values
        onboarding_value = format_with_units(onboarding_value)
        most_recent_value = format_with_units(most_recent_value)
        pr_value = format_with_units(pr_value)

        def format_bold_metric(value, date_phase):
            """ Bold the metric value but keep date/phase normal. """
            return f"$\\bf{{{value}}}$\n{date_phase}" if value != "N/A" else "N/A"

        # **Append the formatted row to the table_data**
        table_data.append([
            metric_label,  # Metric name (already fixed for spacing)
            format_bold_metric(onboarding_value, onboarding_info),
            format_bold_metric(most_recent_value, most_recent_info),
            format_bold_metric(pr_value, pr_info),
        ])

    # **Create the table plot (only once, after loop finishes)**
    fig, ax = plt.subplots(figsize=(12, len(table_data) * 0.6 + 2))  # Adjust height dynamically
    ax.axis("off")  # Remove axes

    # **Now, create the table with finalized `table_data`**
    table = ax.table(
        cellText=table_data,
        colLabels=column_headers,
        loc="center",
        cellLoc="center",
        colColours=["#4f81bd"] * len(column_headers),  # Blue header background
        bbox=[0, 0, 1, 1],  # Cover the entire figure
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.auto_set_column_width(col=list(range(len(column_headers))))

    # Format headers
    for (row, col), cell in table.get_celld().items():
        if row == 0:  # Header row
            cell.set_text_props(weight="bold", color="white", fontsize=9)

    # **Bold only the first column manually (fix metric name bolding)**
    for row in range(1, len(table_data) + 1):
        table.get_celld()[(row, 0)].set_text_props(weight="bold")  # Bold metric name

    # Alternating row colors for readability
    for row, col in table.get_celld():
        if row > 0:  # Skip header row
            if row % 2 == 0:
                table.get_celld()[(row, col)].set_facecolor("#f2f2f2")  # Light gray
            else:
                table.get_celld()[(row, col)].set_facecolor("white")  # White

    # Add title
    ax.set_title(
        "Metric Summary",
        fontsize=16,
        fontweight="bold",
        loc="center",
        pad=20,
    )

    # Save the table to the PDF
    pdf.savefig(fig)
    plt.close(fig)
